# Class 5: APIs

## APIs

- They are very powerful
- Can be used from any programming language
- Not expecting you to use them in your Final Project

## APIs, conceptually

![Diagram showing how online payments work: Expedia talks to Delta, Delta talks to Stripe, Stripe talks to Visa, and Visa talks to Chase](extras/img/apis_conceptually/payments.png)

![Diagram showing how notifications flow through systems](extras/img/apis_conceptually/notifications.png)

![Diagram showing relationship between human languages, programming languages, and APIs](extras/img/apis_conceptually/languages.png)

interactions between systems ↔️

## Ways to get data

Method | How it happens | Pros | Cons
--- | :--- | :--- | :---
**Bulk** | Download, someone hands you a flash drive, etc. | Fast, one-time transfer | Can be large; data gets out of date easily
**APIs** | If organization makes one available | Usually allows some filtering; can always pull latest-and-greatest | Requires network connection for every call; higher barrier to entry (reading documentation); subject to availability and performance of API
**Scraping** | Data only available through a web site, PDF, or doc | You can turn anything into data | Tedious; fragile

## Scraping

Common tools:

- [Beautiful Soup package](https://realpython.com/beautiful-soup-web-scraper-python/)
- [pandas' `read_html()`](https://pandas.pydata.org/docs/user_guide/io.html#html)

_Please pray to the Demo Gods that these all work and there's no profanity_

Pull table from [Wikipedia's list of countries by area](https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area#Countries_and_dependencies_by_area):

In [1]:
import pandas as pd

tables = pd.read_html(
    "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area",
    match="Country / dependency",
)
countries = tables[0]
countries

,Unnamed: 0,Country / dependency,Total in km2 (mi2),Land in km2 (mi2),Water in km2 (mi2),% water,Unnamed: 6
0,–,Earth,"510,072,000 (196,940,000)","148,940,000 (57,506,000)","361,132,000 (139,434,000)",70.8,NaN
1,1,Russia,"17,098,246 (6,601,667)","16,376,870 (6,323,142)","721,380 (278,530)",4.2,[b]
2,–,Antarctica,"14,200,000 (5,480,000)","14,200,000 (5,480,000)",0,0.0,[c]
3,2,Canada,"9,984,670 (3,855,100)","9,093,507 (3,511,021)","891,163 (344,080)",8.9,[d]
4,3/4 [e],China,"9,596,960 (3,705,410)","9,326,410 (3,600,950)","270,550 (104,460)",2.8,[f]
...,...,...,...,...,...,...,...
259,–,Ashmore and Cartier Islands (Australia),5.0 (1.9),5.0 (1.9),0,0.0,[q]
260,–,Coral Sea Islands (Australia),3.0 (1.2),3.0 (1.2),0,0.0,[db]
261,–,Spratly Islands (disputed),2.0 (0.77),2.0 (0.77),0,0.0,[54]
262,194,Monaco,2.0 (0.77),2.0 (0.77),0,0.0,[dc]


### Data is only available if it's available

## API calls in the wild

1. Go to [Candidates page on fec.gov](https://www.fec.gov/data/candidates/?has_raised_funds=true&is_active_candidate=true).
1. Right click and `Inspect`.
   - [More info about opening Developer Tools in various browsers.](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_are_browser_developer_tools#how_to_open_the_devtools_in_your_browser)
1. Go to the `Network` tab and reload.
1. Filter to `XHR`.
1. Click the API call.

We only see this because the tables on [fec.gov](https://fec.gov) are [rendered client-side](https://www.solutelabs.com/blog/client-side-vs-server-side-rendering-what-to-choose-when) using their JSON API. That won't be the case for all tables on all sites.

### Parts of a URL

![URL structure](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/Web_mechanics/What_is_a_URL/mdn-url-all.png)

[source](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL#basics_anatomy_of_a_url)

For APIs:

- Often split into "base URL" + "endpoint"
- Endpoints are like function names: they represent the information you are retrieving or thing you are trying to do
- Parameters are like function arguments:
   - They allow options to be specified
   - Some are required, some are optional
   - They will differ from one endpoint/function to another
- Anchors won't be used

### API documentation

[FEC API](https://api.open.fec.gov/developers/)

### Try it out

1. Visit https://www.fec.gov/data/candidates/
1. [Open Developer Tools](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_are_browser_developer_tools#how_to_open_the_devtools_in_your_browser).
1. Reload the page.
1. In the Network tab's request list:
   1. Filter to Fetch/XHR/AJAX (terminology will differ by browser)
   1. Right-click the API call row.
1. Click `Open in New Tab`. You will see an error.
1. In the URL bar, replace the `api_key` value with `DEMO_KEY`. The URL should therefore contain `api_key=DEMO_KEY`.

You should see a big wall of JSON data.


## API calls from Python

Usually one of two ways:

- A software development kit (SDK) like [sodapy](https://pypi.org/project/sodapy/)
   - Abstracts the details away
   - Not available for all APIs
   - May have limitations
- [The `requests` package](https://docs.python-requests.org/) (nothing to do with 311 requests)

Get Jimmy McMillan's latest candidacy information:


In [2]:
import requests

jimmy = {
    "api_key": "DEMO_KEY",
    "q": "Jimmy McMillan",
    "sort": "-first_file_date",
}
response = requests.get("https://api.open.fec.gov/v1/candidates/", params=jimmy)
data = response.json()
data

{'api_version': '1.0',
 'pagination': {'count': 2,
  'is_count_exact': True,
  'page': 1,
  'pages': 1,
  'per_page': 20},
 'results': [{'active_through': 2016,
   'candidate_id': 'P60016805',
   'candidate_inactive': False,
   'candidate_status': 'N',
   'cycles': [2016, 2018],
   'district': '00',
   'district_number': 0,
   'election_districts': ['00'],
   'election_years': [2016],
   'federal_funds_flag': False,
   'first_file_date': '2015-10-13',
   'has_raised_funds': False,
   'inactive_election_years': None,
   'incumbent_challenge': 'O',
   'incumbent_challenge_full': 'Open seat',
   'last_f2_date': '2015-10-13',
   'last_file_date': '2015-10-13',
   'load_date': '2018-02-17T09:16:20',
   'name': 'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH',
   'office': 'P',
   'office_full': 'President',
   'party': 'REP',
   'party_full': 'REPUBLICAN PARTY',
   'state': 'US'},
  {'active_through': 2012,
   'candidate_id': 'P60003290',
   'candidate_inactive': False,
   'candidate_status': 'N',


### Retrieving nested data

In [3]:
data["results"][0]["name"]

'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH'

### In-class exercise

{% if id == 'nyu' %}_Let's do this together._{% endif %}

Open a new notebook in [{{coding_env_name}}]({{coding_env_url}}), and [geocode](https://en.wikipedia.org/wiki/Address_geocoding) an address from Python using the [Nominatim API](https://nominatim.org/release-docs/develop/api/Search/). Print out the latitude and longitude. Any address is fine:

- Your own
- This building
- etc.

Per [their policies](https://operations.osmfoundation.org/policies/nominatim/#requirements), we'll need to specify a custom [User Agent](https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/User-Agent). Include the following in the call to `requests.get()`:

```python
headers={"user-agent": "Python in-class exercise"}
```

**Hint:** Try getting the API call working in your browser URL bar before calling it in Python.

### Reading into a DataFrame

In [4]:
candidates_with_funds = {
    "api_key": "DEMO_KEY",
    "has_raised_funds": "true",
}
response = requests.get("https://api.open.fec.gov/v1/candidates/", params=candidates_with_funds)
data = response.json()

candidates = data["results"]
pd.DataFrame(candidates)

,active_through,candidate_id,candidate_inactive,candidate_status,cycles,district,district_number,election_districts,election_years,federal_funds_flag,...,incumbent_challenge_full,last_f2_date,last_file_date,load_date,name,office,office_full,party,party_full,state
0,2022,H2CO07170,False,P,"[2022, 2024]",07,7,[07],[2022],False,...,Open seat,2022-08-10,2022-08-10,2023-03-09T10:16:03,"AADLAND, ERIK",H,House,REP,REPUBLICAN PARTY,CO
1,2022,H2UT03280,False,C,[2022],03,3,[03],[2022],False,...,Challenger,2022-03-21,2022-03-21,2022-04-13T21:10:09,"AALDERS, TIM",H,House,REP,REPUBLICAN PARTY,UT
2,2018,S2UT00229,False,P,"[2012, 2014, 2016, 2018, 2020]",00,0,"[00, 00]","[2012, 2018]",False,...,Open seat,2018-04-23,2018-04-23,2019-03-27T16:02:41,"AALDERS, TIMOTHY NOEL",S,Senate,CON,CONSTITUTION PARTY,UT
3,2020,H0TX22260,False,C,[2020],22,22,[22],[2020],False,...,Open seat,2019-10-17,2019-10-17,2020-03-18T21:13:37,"AALOORI, BANGAR REDDY",H,House,REP,REPUBLICAN PARTY,TX
4,1978,H6PA16106,False,P,"[1976, 1978, 1980]",16,16,"[16, 16]","[1976, 1978]",False,...,None,1978-07-05,1978-07-05,2002-03-30T00:00:00,"AAMODT, NORMAN O.",H,House,REP,REPUBLICAN PARTY,PA
5,2012,H2CA01110,False,P,"[2012, 2014, 2016]",01,1,[01],[2012],False,...,Challenger,2012-02-22,2012-02-22,2013-04-26T09:04:30,"AANESTAD, SAMUEL",H,House,REP,REPUBLICAN PARTY,CA
6,2018,H8CO06237,False,C,[2018],06,6,[06],[2018],False,...,Challenger,2017-04-26,2017-04-26,2017-08-01T20:57:28,"AARESTAD, DAVID",H,House,DEM,DEMOCRATIC PARTY,CO
7,2008,P80002926,False,N,"[2006, 2008, 2010, 2012, 2014, 2016]",00,0,[00],[2008],False,...,Open seat,2007-03-13,2007-03-13,2016-11-17T06:10:48,"AARON, LAURA DAVIS",P,President,DEM,DEMOCRATIC PARTY,US
8,2024,H4OR05312,False,C,[2024],05,5,[05],[2024],False,...,Challenger,2023-07-24,2023-07-24,2023-10-16T21:03:02,"AASEN, ANDREW J",H,House,NON,NON-PARTY,OR
9,2024,H2CA30291,False,N,"[2022, 2024]",32,32,"[32, 32]","[2022, 2024]",False,...,Challenger,2022-07-15,2022-07-15,2023-01-12T22:24:01,"AAZAMI, SHERVIN",H,House,DEM,DEMOCRATIC PARTY,CA


## Back to 311 data

From [NYC Open Data Portal dataset page](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/data), click `Export` -> `SODA API` -> `API Docs`.

### Most open data sites have APIs

Often built on platforms that provide them, e.g.

- [NYC Open Data Portal](https://opendata.cityofnewyork.us/) built on [Socrata](https://dev.socrata.com/)
- [data.gov built on CKAN](https://www.data.gov/developers/apis)

### Example: 311 requests from the last week

_The dates shown are from the last time the code was run._

In [5]:
from datetime import datetime, timedelta

now = datetime.utcnow()
now

datetime.datetime(2024, 9, 21, 3, 31, 29, 889839)

In [6]:
start = now - timedelta(weeks=1)
start

datetime.datetime(2024, 9, 14, 3, 31, 29, 889839)

In [7]:
start.isoformat()

'2024-09-14T03:31:29.889839'

Using the [Socrata query language (SoQL)](https://dev.socrata.com/docs/queries/):

In [8]:
data_id = "erm2-nwe9"
in_past_week = {
    "$where": f"created_date > '{start.isoformat()}'",
}

url = f"https://data.cityofnewyork.us/resource/{data_id}.json"
response = requests.get(url, params=in_past_week)
data = response.json()

data

[{'unique_key': '62503694',
  'created_date': '2024-09-20T01:35:56.000',
  'agency': 'NYPD',
  'agency_name': 'New York City Police Department',
  'complaint_type': 'Noise - Residential',
  'descriptor': 'Loud Music/Party',
  'location_type': 'Residential Building/House',
  'incident_zip': '10467',
  'incident_address': '2310 HOLLAND AVENUE',
  'street_name': 'HOLLAND AVENUE',
  'cross_street_1': 'ASTOR AVENUE',
  'cross_street_2': 'WARING AVENUE',
  'intersection_street_1': 'ASTOR AVENUE',
  'intersection_street_2': 'WARING AVENUE',
  'address_type': 'ADDRESS',
  'city': 'BRONX',
  'landmark': 'HOLLAND AVENUE',
  'status': 'In Progress',
  'community_board': '11 BRONX',
  'bbl': '2043480005',
  'borough': 'BRONX',
  'x_coordinate_state_plane': '1021437',
  'y_coordinate_state_plane': '252619',
  'open_data_channel_type': 'PHONE',
  'park_facility_name': 'Unspecified',
  'park_borough': 'BRONX',
  'latitude': '40.859977240527726',
  'longitude': '-73.86556421778361',
  'location': {'la

Like the FEC, Socrata uses their own API to populate the tables when browsing data on sites powered by them.

**At-home exercise:** Try filtering a table on the [NYC Open Data Portal](https://data.cityofnewyork.us/), and find the API calls that makes.

In [9]:
pd.DataFrame(data)

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,...,vehicle_type,resolution_action_updated_date,closed_date,resolution_description,facility_type,taxi_pick_up_location,bridge_highway_name,bridge_highway_segment,taxi_company_borough,road_ramp
0,62503694,2024-09-20T01:35:56.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10467,2310 HOLLAND AVENUE,HOLLAND AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,62500725,2024-09-20T01:35:16.000,NYPD,New York City Police Department,Drinking,In Public,Residential Building/House,11373,56-16 SEABURY STREET,SEABURY STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,62500759,2024-09-20T01:35:00.000,NYPD,New York City Police Department,Illegal Parking,Double Parked Blocking Traffic,Street/Sidewalk,10040,30 DONGAN PLACE,DONGAN PLACE,...,Car,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,62502203,2024-09-20T01:34:37.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10456,1465 WASHINGTON AVENUE,WASHINGTON AVENUE,...,NaN,2024-09-20T01:40:09.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,62498131,2024-09-20T01:34:35.000,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Talking,Street/Sidewalk,10456,1259 GRANT AVENUE,GRANT AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,62499803,2024-09-19T22:22:21.000,NYPD,New York City Police Department,Encampment,N/A,Subway,NaN,NaN,NaN,...,NaN,2024-09-20T02:35:59.000,NaN,NaN,NaN,NaN,R,Entrance,NaN,4 Av & Prospect Expy to Bay Ridge - 95 S
996,62502573,2024-09-19T22:22:21.000,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11232,233 23 STREET,23 STREET,...,NaN,2024-09-19T23:07:19.000,2024-09-19T23:07:16.000,The Police Department issued a summons in resp...,NaN,NaN,NaN,NaN,NaN,NaN
997,62502155,2024-09-19T22:22:02.000,NYPD,New York City Police Department,Illegal Parking,Commercial Overnight Parking,Street/Sidewalk,11235,2456 OCEAN PARKWAY,OCEAN PARKWAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,62500339,2024-09-19T22:22:00.000,DOT,Department of Transportation,Street Light Condition,Street Light Out,NaN,10304,CLINTON STREET,CLINTON STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Coincidence there were exactly 1,000 results?

### Pagination

- Most APIs limit the number of results returned.
- [Socrata defaults to 1,000.](https://dev.socrata.com/docs/queries/limit.html)
- Need to use a loop with parameters like [`$limit`](https://dev.socrata.com/docs/queries/limit.html)+[`$offset`](https://dev.socrata.com/docs/queries/offset.html) (Socrata) or `page`+`per_page` ([FEC](https://api.open.fec.gov/developers/))
   - [`concat()`](https://pandas.pydata.org/pandas-docs/version/1.5/user_guide/merging.html#concatenating-objects) to DataFrame

## Things are going to differ by API

- Endpoints
- Supported parameters
- Response structure
   - [`json_normalize()`](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#normalization) can help
- Quality of documentation
- Helpfulness of errors
- Size/helpfulness of community

Gotta read and experiment.

## [Final Project](https://python-public-policy.afeld.me/en/{{school_slug}}/final_project.html)

- You should have received feedback on your proposal.
- Reminder that it's peer-graded.
   - You should see the notebooks you need to review come through {{lms_name}}.
   - This is an opportunity to see how different people solve different problems.
   - You will lose points if you don't complete your peer grading.
- [Submission](https://python-public-policy.afeld.me/en/{{school_slug}}/final_project.html#submission)

All together, let's make sure our notebooks are properly shared for peer grading:

1. Create a notebook for your Final Project, if you haven't already.
1. [Make it visible to your peers.](https://python-public-policy.afeld.me/en/{{school_slug}}/final_project.html#sharing)

## [Schedule](https://python-public-policy.afeld.me/en/{{school_slug}}/syllabus.html#schedule)